# Instructions

Travail individuel à réaliser par chaque étudiant. Chaque fichier devra ensuite être rassemblé par groupe dans le premier dépôt Git de l'année universitaire, dans un nouveau dossier nommé <code>Computer Vision</code>.

Le nom du fichier doit être le prénom de l'étudiant écrit en minuscules. Par exemple, si l'étudiant s'appelle BOB Toto, le fichier doit être nommé toto.ipynb.

# Détails de l'étudiant
### Nom(s)  : ANDRIAKOTO
### Prénom(s) : Rah-Maesch
### Classe : IMTICIA 4

# Vision par Ordinateur avec Keras/TensorFlow : Un Notebook Pratique et Conceptuel

Ce notebook a pour objectif de vous guider pas à pas dans la création et l'analyse d'un modèle de réseau de neurones convolutif (CNN) appliqué au jeu de données CIFAR-10. Chaque étape est accompagnée d'explications pratiques ainsi que de questions conceptuelles pour renforcer votre compréhension des enjeux théoriques et pratiques de la vision par ordinateur.

## Étape 1 : Introduction et Configuration de l'Environnement

Dans cette étape, nous allons configurer notre environnement de travail et importer les bibliothèques indispensables pour le deep learning et la manipulation de données. Nous vérifions également la version de TensorFlow pour nous assurer que tout fonctionne correctement.

### Explication Pratique
La bonne configuration de l'environnement est cruciale pour garantir la reproductibilité et la stabilité de vos expériences. En particulier, les versions des bibliothèques peuvent influencer le comportement du modèle et sa performance, d'où l'importance de vérifier et documenter ces versions dès le début.

In [ ]:
# Importer les bibliothèques nécessaires
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

print('Version de TensorFlow :', tf.__version__)

Version de TensorFlow : 2.18.0


### Question  1

**Q1 :** Pourquoi est-il essentiel de vérifier la configuration de l'environnement (versions des bibliothèques, dépendances, etc.) avant de développer un modèle de deep learning ?

_Répondez dans une nouvelle cellule Markdown._

En deep learning, on utilise de nombreuses librairies comme TensorFlow, Keras, etc. Il est crucial de vérifier leurs versions car elles peuvent impacter le fonctionnement et les performances du modèle.

Une version différente d'une librairie peut modifier le comportement d'une fonction ou introduire des incompatibilités. Un environnement bien défini assure la reproductibilité des résultats : on peut facilement recréer le même environnement sur une autre machine et obtenir les mêmes résultats. Cela facilite également la collaboration entre chercheurs, car chacun travaille avec les mêmes outils et versions.

Imaginez, si on ne précisait pas les ingrédients d'une recette, on aurait du mal à la refaire à l'identique !

## Étape 2 : Chargement et Prétraitement des Données

Nous allons charger le jeu de données CIFAR-10, composé de 60 000 images couleur réparties en 10 classes. Dans cette étape, nous normalisons les valeurs des pixels afin qu'elles soient comprises entre 0 et 1, et nous transformons les étiquettes en format one-hot pour faciliter le processus de classification.

### Explication Pratique
La normalisation aide à stabiliser et accélérer l'entraînement du modèle en assurant que les valeurs d'entrée ont une échelle comparable. Le one-hot encoding évite que le modèle interprète les étiquettes comme des valeurs numériques ordonnées, ce qui est essentiel pour les problèmes de classification multi-classes.

In [ ]:
# Charger le jeu de données CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normaliser les valeurs des pixels (entre 0 et 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convertir les vecteurs de classes en matrices binaires (one-hot encoding)
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("Forme des données d'entrainement :", x_train.shape)
print("Forme des étiquettes d'entraînement :", y_train.shape)

### Question 2

**Q2 :** Expliquez comment la normalisation des pixels et le one-hot encoding des étiquettes contribuent chacun à la stabilité et à l'efficacité de l'entraînement d'un modèle de deep learning.

_Répondez dans une nouvelle cellule Markdown._

La normalisation des pixels, en les ramenant entre 0 et 1, évite des problèmes d'échelle qui pourraient perturber l'apprentissage.

Si les valeurs des pixels sont très différentes (par exemple, entre 0 et 255), certaines caractéristiques pourraient avoir plus d'influence que d'autres, ce qui rend l'apprentissage plus difficile.

Le one-hot encoding, lui, transforme les étiquettes en vecteurs binaires. Par exemple, si on a 3 classes (chat, chien, oiseau), l'étiquette "chat" serait représentée par le vecteur [1, 0, 0].

Cela permet au modèle de traiter chaque catégorie de manière indépendante, sans créer de hiérarchie artificielle entre elles (le modèle ne va pas penser que "chien" est "plus grand" que "chat" par exemple).

En gros, on met les données dans un format optimal pour que le modèle apprenne efficacement.

## Étape 3 : Exploration et Visualisation des Données

Avant de construire le modèle, il est important d'explorer et de visualiser les données. Nous affichons ainsi un échantillon d'images du jeu de données pour mieux comprendre leur contenu et la distribution des classes.

### Explication Pratique
La visualisation des données permet d'identifier d'éventuelles anomalies, comme des classes sous-représentées ou des images bruitées, et de décider si des techniques d'augmentation de données ou de prétraitement supplémentaires sont nécessaires.

In [ ]:
# Afficher quelques images du jeu de données d'entraînement
noms_classes = ['avion', 'automobile', 'oiseau', 'chat', 'cerf',
               'chien', 'grenouille', 'cheval', 'navire', 'camion']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(noms_classes[y_train[i].argmax()])
plt.show()

### Question 3

**Q3 :** D'après la visualisation, discutez de l'impact potentiel d'une distribution inégale des classes ou de la présence d'images de mauvaise qualité sur la performance d'un modèle de classification. Quelles stratégies pourraient être mises en place pour pallier ces problèmes ?

_Répondez dans une nouvelle cellule Markdown._

Si certaines classes sont sur-représentées dans le jeu de données, le modèle risque de se spécialiser sur celles-ci et de mal généraliser aux autres classes.

Par exemple, si on a beaucoup plus d'images de chats que de chiens, le modèle sera très bon pour reconnaître les chats, mais moins bon pour les chiens. Des images de mauvaise qualité (floues, bruitées, etc.) peuvent aussi perturber l'apprentissage, car le modèle aura du mal à en extraire des caractéristiques pertinentes.

Pour y remédier, on peut rééquilibrer les classes en utilisant des techniques comme le sur-échantillonnage (augmenter le nombre d'exemples pour les classes minoritaires) ou le sous-échantillonnage (réduire le nombre d'exemples pour les classes majoritaires).

On peut aussi améliorer les images en utilisant des techniques de traitement d'image (réduction du bruit, augmentation de la netteté, etc.).

Enfin, on peut utiliser des techniques de régularisation pour rendre le modèle plus robuste aux variations des données et éviter le surapprentissage. L'idée est d'avoir un jeu de données équilibré et représentatif du problème.

## Étape 4 : Construction du Modèle CNN

Nous allons construire un réseau de neurones convolutif (CNN) pour extraire des caractéristiques hiérarchiques des images. Ce modèle se compose de plusieurs blocs de convolution suivis de couches de pooling et se termine par des couches entièrement connectées pour la classification.

### Explication Pratique
Les couches de convolution permettent au modèle de détecter des motifs locaux (comme les contours ou les textures), tandis que les couches de pooling réduisent la dimensionnalité, ce qui diminue la charge computationnelle et aide à rendre le modèle plus robuste aux translations. Le dropout, quant à lui, est une technique de régularisation qui aide à prévenir le surapprentissage en désactivant aléatoirement certains neurones pendant l'entraînement.

In [ ]:
# Construire le modèle CNN
model = models.Sequential()

# Bloc de convolution 1 : 32 filtres, taille 3x3, activation ReLU
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))

# Bloc de convolution 2 : 64 filtres
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Bloc de convolution 3 : 64 filtres
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Aplatir les sorties et ajouter des couches entièrement connectées
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

### Question 4

**Q4 :** Décrivez le rôle de chaque composant du CNN (couches de convolution, pooling et dropout) et expliquez comment ils interagissent pour permettre au modèle d'extraire des caractéristiques pertinentes des images.

_Répondez dans une nouvelle cellule Markdown._

Les couches de convolution agissent comme des détecteurs de motifs locaux, comme des contours, des textures ou des formes simples. Chaque couche de convolution applique un ensemble de filtres à l'image d'entrée pour extraire ces motifs.

Le pooling réduit la dimensionnalité des données en regroupant les pixels voisins et en conservant seulement la valeur maximale ou moyenne. Cela simplifie le traitement de l'information et rend le modèle moins sensible aux petites variations dans les images, comme des légers déplacements ou des rotations.

Le dropout, lui, désactive aléatoirement des neurones pendant l'apprentissage, ce qui évite le surapprentissage. En forçant le modèle à apprendre des représentations plus robustes et moins dépendantes de neurones individuels, le dropout améliore sa capacité de généralisation.


En combinant ces couches, le CNN apprend à extraire des caractéristiques de plus en plus abstraites pour classifier les images. Par exemple, les premières couches de convolution pourraient détecter des contours, les couches suivantes pourraient détecter des formes simples, et les dernières couches pourraient détecter des objets complexes.

## Étape 5 : Compilation et Entraînement du Modèle

Nous allons maintenant compiler le modèle en choisissant un optimiseur, une fonction de perte ainsi que des métriques d'évaluation. Ensuite, nous entraînons le modèle sur les données d'entraînement en réservant une partie des données pour la validation.

### Explication Pratique
La compilation configure le processus d'apprentissage, notamment la manière dont les poids seront ajustés via la rétropropagation. Le choix de l'optimiseur (ici, Adam) et la définition des hyperparamètres (comme le taux d'apprentissage et la taille du batch) influencent grandement la vitesse de convergence et la qualité finale du modèle.

In [ ]:
# Compiler le modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(x_train, y_train, epochs=10, batch_size=64,
                    validation_split=0.2)

### Question 5

**Q5 :** Quels sont les effets d'un choix inadapté d'hyperparamètres (comme le taux d'apprentissage ou la taille du batch) sur l'entraînement d'un réseau de neurones ? Expliquez en quoi un optimiseur bien configuré est crucial pour la convergence du modèle.

_Répondez dans une nouvelle cellule Markdown._

Des hyperparamètres mal choisis, comme un taux d'apprentissage trop élevé ou une taille de batch trop petite, peuvent empêcher le modèle de converger vers une solution optimale. Un taux d'apprentissage trop élevé peut faire "sauter" l'optimisation au-dessus des minima locaux, tandis qu'un taux trop faible peut la rendre très lente.

La taille du batch détermine le nombre d'exemples utilisés pour chaque mise à jour des poids. Une taille trop petite rend l'apprentissage plus bruité, tandis qu'une taille trop grande peut nécessiter beaucoup de mémoire.

L'optimiseur, lui, guide l'apprentissage en ajustant les poids du modèle en fonction des gradients de la fonction de perte. S'il est mal configuré, l'apprentissage peut être lent, instable, voire diverger.

C'est comme si on essayait de trouver le sommet d'une montagne en faisant des pas trop grands ou trop petits : il faut trouver le bon équilibre pour progresser efficacement.

## Étape 6 : Évaluation du Modèle

Après l'entraînement, nous évaluons notre modèle sur le jeu de test afin de mesurer sa capacité de généralisation sur des données inédites. Les métriques telles que la perte et la précision nous aident à quantifier la performance globale du modèle.

### Explication Pratique
L'évaluation sur un jeu de test indépendant permet de détecter un éventuel surapprentissage (overfitting). Si le modèle présente une bonne performance sur l'entraînement mais une performance médiocre sur le test, cela indique qu'il n'a pas suffisamment généralisé, ce qui peut nécessiter des ajustements comme plus de régularisation ou des techniques d'augmentation de données.

In [ ]:
# Évaluer le modèle sur le jeu de test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Précision sur le jeu de test :', test_acc)

### Question  6

**Q6 :** Que nous indiquent la perte et la précision obtenues lors de l'évaluation sur le jeu de test ? Quels ajustements pourriez-vous envisager si vous observez un écart significatif entre les performances sur l'entraînement et le test ?

_Répondez dans une nouvelle cellule Markdown._

La perte (loss) et la précision (accuracy) sur le jeu de test reflètent la capacité du modèle à généraliser sur de nouvelles données, qu'il n'a jamais vues pendant l'entraînement.

La perte mesure l'erreur du modèle dans ses prédictions, tandis que la précision représente le pourcentage de prédictions correctes. Un écart important entre les performances sur l'entraînement et le test suggère un surapprentissage : le modèle a "mémorisé" les données d'entraînement et n'est pas capable de généraliser sur des données inconnues.

Pour y remédier, on peut augmenter la quantité de données d'entraînement, ce qui permet au modèle d'apprendre des caractéristiques plus générales.

On peut aussi régulariser le modèle en utilisant des techniques comme le dropout ou la régularisation L2, ce qui limite sa complexité et évite le surapprentissage.

Enfin, on peut simplifier l'architecture du modèle en réduisant le nombre de couches ou de neurones, ce qui le rend moins susceptible de surapprendre.

L'objectif est d'obtenir un modèle performant non seulement sur les données d'entraînement, mais aussi sur des données inconnues.

## Étape 7 : Prédictions et Visualisation des Résultats

Nous allons utiliser le modèle entraîné pour prédire les classes des images du jeu de test. La visualisation des résultats nous permet de comparer les étiquettes prédites aux étiquettes réelles et d'identifier les erreurs potentielles.

### Explication Pratique
La visualisation aide à comprendre qualitativement comment le modèle se comporte face à différentes images. Cela permet d'identifier si certaines classes sont systématiquement mal prédites ou si le modèle confond certaines catégories, ouvrant ainsi la voie à des améliorations ultérieures (par exemple, via l'augmentation de données ou des ajustements de l'architecture).

In [ ]:
# Faire des prédictions sur le jeu de test
predictions = model.predict(x_test)

# Fonction pour afficher l'image avec les étiquettes prédites et réelles
def afficher_image(i, predictions_array, etiquette_vraie, img):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)

    etiquette_predite = np.argmax(predictions_array)
    etiquette_vraie = np.argmax(etiquette_vraie)

    couleur = 'blue' if etiquette_predite == etiquette_vraie else 'red'
    plt.xlabel(f"Prédit : {noms_classes[etiquette_predite]} (Vrai : {noms_classes[etiquette_vraie]})", color=couleur)

# Afficher quelques images de test avec leurs prédictions
nb_lignes = 5
nb_colonnes = 3
nb_images = nb_lignes * nb_colonnes
plt.figure(figsize=(2 * nb_colonnes, 2 * nb_lignes))
for i in range(nb_images):
    plt.subplot(nb_lignes, nb_colonnes, i+1)
    afficher_image(i, predictions[i], y_test[i], x_test[i])
plt.tight_layout()
plt.show()

### Question 7

**Q7 :** Après avoir examiné les prédictions, identifiez et discutez des stratégies conceptuelles (par exemple, l'augmentation de données, le raffinement de l'architecture ou l'ajustement des hyperparamètres) qui pourraient améliorer la robustesse et la précision du modèle.

_Répondez dans une nouvelle cellule Markdown._

Pour améliorer le modèle, on peut envisager d'augmenter la diversité des données d'entraînement en utilisant des techniques d'augmentation de données comme des rotations, des translations, ou des changements de luminosité.

Cela permet au modèle d'apprendre des caractéristiques plus robustes et moins sensibles aux variations des images. On peut aussi affiner l'architecture du CNN en ajoutant des couches de convolution pour apprendre des caractéristiques plus complexes, en modifiant le nombre de filtres ou la taille des filtres pour ajuster la capacité du modèle à détecter différents types de motifs, ou en expérimentant avec différentes fonctions d'activation pour influencer la non-linéarité du modèle et sa capacité d'apprentissage.

Enfin, on peut optimiser les hyperparamètres comme le taux d'apprentissage, la taille du batch, ou le nombre d'époques, en utilisant des techniques de recherche d'hyperparamètres comme la recherche par grille ou la recherche aléatoire. Il s'agit d'un processus itératif qui vise à affiner le modèle pour qu'il soit plus performant et plus généralisable.

## Étape 8 : Conclusion et Travaux Futurs

Dans ce notebook, nous avons :
- Configuré l'environnement et importé les bibliothèques nécessaires
- Chargé et prétraité le jeu de données CIFAR-10
- Exploré et visualisé les données
- Construit, compilé et entraîné un modèle CNN
- Évalué le modèle et visualisé ses prédictions

### Explication Pratique
Ce pipeline offre une approche complète, à la fois pratique et conceptuelle, pour la mise en œuvre d'un modèle de vision par ordinateur. Pour aller plus loin, vous pouvez explorer des architectures plus complexes, appliquer des techniques d'augmentation de données ou encore expérimenter avec différents optimisateurs afin de mieux comprendre l'impact de chacun sur la performance du modèle.